<a href="https://colab.research.google.com/github/cshyang/coding-projects/blob/master/SWVL_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem 4: Python or R Only.
#####Treatment and Control splits are crucial for an effective and fair A-B test. Groups should be as identical as possible. Perform the following in order in your preferred programming language:

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('SWVL_Growth_Hacker_Case_DataSets').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame(rows)
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df.head()

,booking_id,user_id,booking_date_stamp,pick_up_time,drop_off_time,pickup_dropoff_distance_km,ride_date,total_price,status,cancellation_date,cancellation_reason,promo_deduction,package_discount,cash_payment,credit_card_payment,promotion,rating
1,85db66ce82d06424d07bab3a85890243,35116f9c68abf857e7540dc644ea48ec,2021-05-18 15:04:04.621000+00:00,2021-05-18 15:26:09+00:00,2021-05-18 16:33:30+00:00,37.853,2021-05-18 15:15:00+00:00,117,completed,,,23.4,,90.35,0,133ef4102276c622027e5c4ec8abf913,4
2,d9c7b92be26d46d38dfdfeefb8b3bb53,4f0758ce8781c50f6fa01ecf88e7ad3b,2021-03-24 15:11:27.571000+00:00,2021-03-24 15:38:35+00:00,2021-03-24 16:33:00+00:00,22.797,2021-03-24 14:55:00+00:00,78,completed,,,23.4,,42.9,0,03f92922f9119c18676930fc013b1d0d,5
3,c86861eeb58e6844ec864a85d5b9664c,1ecb152a0bec29d11c04a3011aed248b,2021-05-20 16:29:32.956000+00:00,2021-05-20 19:00:00+00:00,2021-05-20 20:01:08+00:00,32.543,2021-05-20 19:00:00+00:00,48.75,completed,,,12.1875,,34.6125,0,261748a29e58805a2a5207d46c1b380c,5
4,7b613cb35ef01e0c180e403923675fa1,68740e23bfa17315823c15499beeed2b,2021-06-02 08:21:03.069000+00:00,2021-06-02 13:27:14+00:00,2021-06-02 14:26:39+00:00,9.717,2021-06-02 13:10:00+00:00,29.25,completed,,,4.3875,,24.7,0,b3b55796707b261724cad1f01216ba71,5
5,b326facd4dbe726b2d556b348c73ac86,f57ba723e8f2c6a014bd8ad82c860e0b,2021-04-26 23:48:06.941000+00:00,2021-04-27 10:38:50+00:00,2021-04-27 11:39:10+00:00,30.499,2021-04-27 10:30:00+00:00,97.5,completed,,,48.75,,48.75,0,7e44959a8aefd208f2dfc2d8b3979549,5


a)Read your booking data and filter all users who completed at least 1 booking in the last 30 days.

c) Calculate how many bookings completed by each user in the last 30 days.

In [2]:
#import pandasql to run sql query on pandas dataframe
!pip install pandasql
import pandasql as ps

q = """
        select 
          user_id
          , sum(1) as last_30days_bookings
          , max(date(booking_date_stamp)) as last_booking_date
        from 
          df 
        where 
          status = 'completed' 
        and 
          date(booking_date_stamp) >= date('2021-05-07')
        and
          date(booking_date_stamp) <= date('2021-06-06')
        group by
          1
    """

agg = ps.sqldf(q)
agg.head()

  Created wheel for pandasql: filename=pandasql-0.7.3-cp37-none-any.whl size=26784 sha256=9d00786271c25febb1231d771cf5a210c6f564ba6a85d126df613e7a141a592b
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


,user_id,last_30days_bookings,last_booking_date
0,000c7257a5ea078a05e0c7716e4c4522,1,2021-05-24
1,00111f083d1ef214b82118d04bfd9fa2,6,2021-05-17
2,00140a6c46900e2c34bb5cf89f61df7d,5,2021-06-06
3,0054850a1acb68ff3785390c184ea841,1,2021-06-04
4,00648f45109d7885f944a8659df943a8,1,2021-05-13


b) Calculate the days passed since the last completed booking for each user, let’s call this inactivity days.

In [17]:
#calculate days_since_last_order

agg['current_date'] = pd.to_datetime('today').strftime("%Y-%m-%d")
agg[['current_date', 'last_booking_date']] = agg[['current_date', 'last_booking_date']].apply(pd.to_datetime)
agg['days_since_last_order'] = agg['current_date'] - agg['last_booking_date']

agg.head()

,user_id,last_30days_bookings,last_booking_date,current_date,days_since_last_order
0,000c7257a5ea078a05e0c7716e4c4522,1,2021-05-24,2021-06-13,20 days
1,00111f083d1ef214b82118d04bfd9fa2,6,2021-05-17,2021-06-13,27 days
2,00140a6c46900e2c34bb5cf89f61df7d,5,2021-06-06,2021-06-13,7 days
3,0054850a1acb68ff3785390c184ea841,1,2021-06-04,2021-06-13,9 days
4,00648f45109d7885f944a8659df943a8,1,2021-05-13,2021-06-13,31 days


d) Assign a frequency segment to your selected users: HV 5+ bookings, MV 2-4 bookings and LV 1-2 bookings. Eg: if Mike did 6 completed bookings in the last 30 days we will assign HV to him, if he did 1 booking LV etc.

In [11]:
# label users in to HV, MV and LV based on last_30days_bookings

import numpy as np

agg["segment"] = np.select(
    [
     agg["last_30days_bookings"] >=5,
     agg["last_30days_bookings"].between(3, 4, inclusive=True),
     agg["last_30days_bookings"].between(1, 2, inclusive=True)
    ],
    [
     'HV',
     'MV',
     'LV'
    ],
    default = 'Unknown'
)

agg.groupby("segment",as_index=False)["user_id"].count()

,segment,user_id
0,HV,316
1,LV,1508
2,MV,341


e) Write a code piece as you will to split these users into 2 groups.

In [12]:
#Stratify split using sklear train_test_split

from sklearn.model_selection import train_test_split

y = agg.pop('segment').to_frame()
X = agg

X_test, X_control, y_test, y_control = train_test_split(
        X, y,stratify=y, test_size=0.5)

treatment = pd.concat([X_test, y_test], axis=1)
control = pd.concat([X_control, y_control], axis=1)


,last_30days_bookings,days_since_last_order
count,1082.000000,1082
mean,2.702403,19 days 02:07:45.804066543
std,3.149699,9 days 13:22:25.696034035
min,1.000000,7 days 00:00:00
25%,1.000000,11 days 00:00:00
50%,2.000000,17 days 00:00:00
75%,3.000000,28 days 00:00:00
max,38.000000,37 days 00:00:00


**f) Check and show that these groups are similar by frequency segment (c) and inactivity days(b) .  Hint: provide and compare summary statistics like, count, mean,median etc.**

In [18]:
# compare the statistic and segment size between treatment and control groups

display(control.describe() )
display(treatment.describe())
display(control.groupby("segment",as_index=False)["user_id"].count() )
display(treatment.groupby("segment",as_index=False)["user_id"].count() )

,last_30days_bookings,days_since_last_order
count,1083.000000,1083
mean,2.637119,19 days 04:24:35.900277008
std,2.803547,9 days 06:51:58.116483661
min,1.000000,7 days 00:00:00
25%,1.000000,11 days 00:00:00
50%,2.000000,18 days 00:00:00
75%,3.000000,26 days 00:00:00
max,25.000000,37 days 00:00:00


,last_30days_bookings,days_since_last_order
count,1082.000000,1082
mean,2.702403,19 days 02:07:45.804066543
std,3.149699,9 days 13:22:25.696034035
min,1.000000,7 days 00:00:00
25%,1.000000,11 days 00:00:00
50%,2.000000,17 days 00:00:00
75%,3.000000,28 days 00:00:00
max,38.000000,37 days 00:00:00


,segment,user_id
0,HV,158
1,LV,754
2,MV,171


,segment,user_id
0,HV,158
1,LV,754
2,MV,170


*treatment and control groups have very similar means and standard deviation for last_30days_bookings, and have identical numbers of users in each segment.*